##  準備

In [1]:
from pathlib import Path
from bs4 import BeautifulSoup
import json
import os
import re
import time
import shutil
import logging
import pandas as pd
!pip install selenium
!pip install beautiful requests
!pip install BeautifulSoup4
from selenium import webdriver
from selenium.webdriver.support.select import Select
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from time import sleep
from selenium.common.exceptions import UnexpectedAlertPresentException
from selenium.webdriver.common.alert import Alert

pd.set_option('display.max_rows', 100)

  Using cached urllib3-1.26.16-py2.py3-none-any.whl (143 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
anaconda-project 0.10.1 requires ruamel-yaml, which is not installed.
requests 2.22.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.16 which is incompatible.
cookiecutter 1.7.2 requires requests>=2.23.0, but you have requests 2.22.0 which is incompatible.
  Using cached urllib3-1.25.11-py2.py3-none-any.whl (127 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.16
    Uninstalling urllib3-1.26.16:
      Successfully uninstalled urllib3-1.26.16
ERROR: pip's dependency resolv

In [7]:

pip install chromedriver-binary-auto

Note: you may need to restart the kernel to use updated packages.


In [8]:
from selenium import webdriver
from selenium.webdriver.support.select import Select
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from time import sleep
from selenium.common.exceptions import UnexpectedAlertPresentException
from selenium.webdriver.common.alert import Alert


In [3]:
hard_coded_colleage_list =[
    {"univ_name":"大阪観光大学","url":"https://cpweb.tourism.ac.jp/public/web/Syllabus/WebSyllabusKensaku/UI/WSL_SyllabusKensaku.aspx"}]

In [4]:
#検索結果の表示ページを切り替える
def nexe_page(browser):
    next_button = browser.find_element(By.LINK_TEXT, '次ページ＞')   
    next_button.click()


def is_next_page_exist(browser):
    next_button = browser.find_element(By.LINK_TEXT, '次ページ＞')   
    if(next_button.get_attribute("disabled") == "true"):
        print("this is the last page.")
        return False
    else:
        return True

def record_links(browser,file_name):
    main_table = browser.find_element(By.CLASS_NAME, 'last-table')
    tr_elements = main_table.find_elements(By.TAG_NAME, 'tr')
    for tr_element in tr_elements:
    # <input>要素を取得します
        try:
            input_element = tr_element.find_element(By.TAG_NAME, 'input')
        except:
            #ヘッダー行の回避
            continue
        raw_code = input_element.get_attribute("onClick")
        start_index = raw_code.find("win = window.open('") + 1
        end_index = raw_code.find("','WSL_SyllabusSanshoaspx")
        raw_code = input_element.get_attribute("onClick")
        start_index = raw_code.find("win = window.open('") + 1
        end_index = raw_code.find("','WSL_SyllabusSanshoaspx")
        substring = raw_code[start_index:end_index]
        key = substring.replace("in = window.open('", "")
        access_url = base_url + key
        with open(file_name, 'a') as f:
            f.write(access_url+"\n")


In [10]:
def extract_jp_from_string(text):
    match = re.search(r'\.jp', text)
    if match:
        jp_index = match.start() + 3  # 「.jp」の後ろの文字のインデックス
        jp_part = text[:jp_index]  # 「.jp」までの部分を抜き出す
        return jp_part
    else:
        return None


def append_line_to_file(filename, line):
    with open(filename, 'a') as file:
        file.write(line + '\n')

        
options = webdriver.ChromeOptions()
options.binary_location = "/Applications/Google Chrome.app/Contents/MacOS/Google Chrome"
chrome_driver_binary = "./chromedriver"
for column in hard_coded_colleage_list:
    new_dir_path = column['univ_name']
    new_record_path = './'+column['univ_name']+"/"+column['univ_name']+"_links.txt"
    search_url = column['url']
    base_url = extract_jp_from_string( column['url'])
    if not os.path.exists(new_dir_path):
        os.mkdir(new_dir_path)
    if not os.path.exists(new_record_path):
        f = open(new_record_path,'w')
        f.write("")
        f.close()
    browser1 =  webdriver.Chrome(chrome_driver_binary, chrome_options=options)
    browser1.get(search_url)
    sleep(5)
    search_button = browser1.find_element(By.NAME,'btnSearch')
    search_button.click()
    sleep(5)
    record_links(browser1,new_record_path)
    while(is_next_page_exist(browser1)):
        nexe_page(browser1)
        record_links(browser1,new_record_path)
    browser1.close()

/var/folders/_q/5b0cfy_d2ks1qlbxzgflbk880000gn/T/ipykernel_17371/2335857335.py:30: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser1 =  webdriver.Chrome(chrome_driver_binary, chrome_options=options)
/var/folders/_q/5b0cfy_d2ks1qlbxzgflbk880000gn/T/ipykernel_17371/2335857335.py:30: DeprecationWarning: use options instead of chrome_options
  browser1 =  webdriver.Chrome(chrome_driver_binary, chrome_options=options)


this is the last page.


In [12]:
import os
import glob
import random
import string
# カレントディレクトリのパスを取得
current_directory = os.getcwd()

# カレントディレクトリ内のフォルダを取得
folders = [f for f in os.listdir(current_directory) if os.path.isdir(os.path.join(current_directory, f))]

pattern = r'P1=(.*?)&P2'

# フォルダの名前を表示
for folder_path in folders:
    file_paths = glob.glob(os.path.join(folder_path, "*.txt"))
    with open(file_paths[0], "r") as file:
        for line in file:
        # 行の処理を行うためのコードを追加
            response = requests.get(line)
            html = response.text
            match = re.search(pattern, line)
            if match:
                extracted_text = match.group(1)
                file_name = './' + folder_path + "/" + extracted_text + '.html'
            else:
                file_name = './'.join(random.choices(string.ascii_letters + string.digits, k=10)) + '.html'
            print(file_name)
            sleep(10)
            with open(r""+file_name, 'w', encoding='utf-8') as f:
                f.write(html)
                print(file_name)

./大阪観光大学/ST15001.html
./大阪観光大学/ST15001.html
./大阪観光大学/ST15001.html
./大阪観光大学/ST15001.html
./大阪観光大学/ST15006.html
./大阪観光大学/ST15006.html
./大阪観光大学/ST20001.html
./大阪観光大学/ST20001.html
./大阪観光大学/ST20002.html
./大阪観光大学/ST20002.html
./大阪観光大学/ST20003.html
./大阪観光大学/ST20003.html
./大阪観光大学/ST20004.html
./大阪観光大学/ST20004.html
./大阪観光大学/ST20005.html
./大阪観光大学/ST20005.html
./大阪観光大学/ST20006.html
./大阪観光大学/ST20006.html
./大阪観光大学/ST20099.html
./大阪観光大学/ST20099.html
./大阪観光大学/ST25001.html
./大阪観光大学/ST25001.html
./大阪観光大学/ST25002.html
./大阪観光大学/ST25002.html
./大阪観光大学/ST25003.html
./大阪観光大学/ST25003.html
./大阪観光大学/ST25004.html
./大阪観光大学/ST25004.html
./大阪観光大学/ST25005.html
./大阪観光大学/ST25005.html
./大阪観光大学/ST25006.html
./大阪観光大学/ST25006.html
./大阪観光大学/ST25013.html
./大阪観光大学/ST25013.html
./大阪観光大学/ST25014.html
./大阪観光大学/ST25014.html
./大阪観光大学/ST25015.html
./大阪観光大学/ST25015.html
./大阪観光大学/ST25016.html
./大阪観光大学/ST25016.html
./大阪観光大学/ST25099.html
./大阪観光大学/ST25099.html
./大阪観光大学/ST30001.html
./大阪観光大学/ST30001.html
./大阪観光大学/ST30002.html
./大阪観光大学/S